In [1]:
import os
import argparse
import pandas as pd
import numpy as np
import pickle
from utils_test import *

In [2]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [3]:
from keras.models import load_model
from theano.ifelse import ifelse #added this

Using Theano backend.


In [4]:
name = 'helpdesk'
args = {
    'inputdir': '../input/{}/'.format(name),   
    'outputdir': './output_files/{}/'.format(name),
    'modelname': 'model_77-1.51.h5',
}

args = argparse.Namespace(**args)

In [5]:
test = pd.read_csv(args.inputdir+'test.csv')
test = transformDf(test)

In [6]:
with open(args.inputdir+'parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [7]:
with open(args.inputdir+'preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [8]:
model = load_model(args.outputdir+args.modelname)

In [9]:
test_groupByCase = test.groupby(['CaseID'])

#get features all data
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

#get output all data
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

# Check longest case --> suffix: 2 to 7
test_len = findLongestLength(test_groupByCase)
test_len - 5

8

In [10]:
def get_remaining_trace_accuracy(true_labels, pred_probabilities, targetchartoindice):
    true_trace, pred_trace = getRemaingTrace(true_labels, pred_probabilities, targetchartoindice)
    count=0.0
    for i in range(len(true_trace)):
        if true_trace[i] == pred_trace[i]:
            count+=1
    acc = count/len(true_trace)
    print 'Accuracy of remaining trace: {}%'.format(acc*100)

In [11]:
def evalAct(true_label, pred_prob, targetchartoindice):
    #for log-loss: get probabilities
    gt_a = one_hot_encode(true_label, targetchartoindice)
    log_loss = metrics.log_loss(gt_a, pred_prob)
    print 'Log-loss: {}'.format(log_loss)
    
    #for accuracy: get labels
    gt_a_label = true_label
    pred_a_label = getLabel(pred_prob, targetchartoindice)
    acc = metrics.accuracy_score(gt_a_label, pred_a_label)
    print 'Accuracy: {}%'.format(acc*100)
    acc_3 = get_top3_accuracy(pred_prob, gt_a_label, targetchartoindice)
    print 'Top 3 accuracy: {}%'.format(acc_3*100)

In [12]:
def evalTime(true_time, pred_time, divisor):
    #get time
    gt_t = true_time
    pred_t = inverseTime(pred_time, divisor)
    
    mse = metrics.mean_squared_error(gt_t, pred_t)
    mae = metrics.mean_absolute_error(gt_t, pred_t)
    median = metrics.median_absolute_error(gt_t, pred_t)
    
    print 'Mean Squared Error: {0}s\t| {1} days'.format(mse, mse/86400)
    print 'Mean Absolute Error: {0}s\t| {1} days'.format(mae, mae/86400)
    print 'Median Absolute Error: {0}s\t| {1} days'.format(median, median/86400)

In [13]:
# DL distance
#distance.nlevenshtein(gt_a_label, pred_a_label) #0.1731066460587326

#damerau_levenshtein_distance(gt_a_label, pred_a_label) #784

#distance.jaccard(gt_a_label, pred_a_label) #0.4444444444444444

In [14]:
def getSuffix(suffix, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t):
    sentences_2 = []
    sentences_t_2 = []
    sentences_t2_2 = []
    sentences_t3_2 = []
    sentences_t4_2 = []
    next_chars_2 = []
    next_chars_t_2 = []

    for i in range(len(sentences)):
        if len(sentences[i]) == suffix:
            sentences_2.append(sentences[i])
            sentences_t_2.append(sentences_t[i])
            sentences_t2_2.append(sentences_t2[i])
            sentences_t3_2.append(sentences_t3[i])
            sentences_t4_2.append(sentences_t4[i])
            next_chars_2.append(next_chars[i])
            next_chars_t_2.append(next_chars_t[i])
    return sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2

# All data

## Predict

In [15]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [16]:
pred[0].shape, pred[1].shape

((4529, 10), (4529, 1))

## Evaluate

### Activity

In [17]:
get_remaining_trace_accuracy(next_chars, pred_a, targetchartoindice)

Accuracy of remaining trace: 59.9369085174%


In [18]:
evalAct(next_chars, pred_a, targetchartoindice)

Log-loss: 0.556910028043
Accuracy: 82.5568558181%
Top 3 accuracy: 99.1168028262%


### Time

In [19]:
evalTime(next_chars_t, pred_t, divisor)

Mean Squared Error: 2.40264211257e+11s	| 2780835.77844 days
Mean Absolute Error: 194805.482155s	| 2.2546930805 days
Median Absolute Error: 16509.8847656s	| 0.191086629232 days


# Suffix

## Predict

In [36]:
max_suffix = 11

In [76]:
for i in range(2, max_suffix+1):
    print '----------Suffix {}------------'.format(i)
    sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 = getSuffix(i, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)
    X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)
    pred = model.predict(X_test, verbose=0)
    pred_a = pred[0]
    pred_t = pred[1]
    print 'Evaluation of Activity'
    evalAct(next_chars_2, pred[0], targetchartoindice)
    print 'Evaluation of Time'
    evalTime(next_chars_t_2, pred_t, divisor)
    print '\n'
    #Add gt and pred of each suffix
    if i == 2:
        full_gt_a = next_chars_2
        full_pred_a = pred_a
        full_gt_t = next_chars_t_2
        full_pred_t = pred_t
    else:
        full_gt_a += next_chars_2
        full_pred_a = np.concatenate((full_pred_a, pred_a), axis=0)
        full_gt_t += next_chars_t_2
        full_pred_t = np.concatenate((full_pred_t, pred_t), axis=0)

----------Suffix 2------------
Evaluation of Activity
Log-loss: 0.664227357496
Accuracy: 77.6813880126%
Top 3 accuracy: 98.738170347%
Evaluation of Time
Mean Squared Error: 3.49056102713e+11s	| 4040001.18881 days
Mean Absolute Error: 277457.405924s	| 3.21131256856 days
Median Absolute Error: 27724.8496094s	| 0.320889463071 days


----------Suffix 3------------
Evaluation of Activity
Log-loss: 0.4838520479
Accuracy: 84.25%
Top 3 accuracy: 99.4166666667%
Evaluation of Time
Mean Squared Error: 1.95771055972e+11s	| 2265868.70338 days
Mean Absolute Error: 154406.375571s	| 1.78711082837 days
Median Absolute Error: 135.120529175s	| 0.0015638950136 days


----------Suffix 4------------
Evaluation of Activity
Log-loss: 0.472546177274
Accuracy: 84.7191011236%
Top 3 accuracy: 99.5505617978%
Evaluation of Time
Mean Squared Error: 1.19563640635e+11s	| 1383838.43327 days
Mean Absolute Error: 99842.7480553s	| 1.15558736175 days
Median Absolute Error: 648.383666992s	| 0.00750444059019 days


---------

In [77]:
evalAct(full_gt_a, full_pred_a, targetchartoindice)

Log-loss: 0.547011715111
Accuracy: 82.0552147239%
Top 3 accuracy: 99.1717791411%


In [78]:
evalTime(full_gt_t, full_pred_t, divisor)

Mean Squared Error: 2.42946213353e+11s	| 2811877.46937 days
Mean Absolute Error: 191403.711914s	| 2.21532073974 days
Median Absolute Error: 9745.64794922s	| 0.112796851264 days


# Test

## Predict

In [98]:
sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 =  getSuffix(10, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)

In [101]:
len(sentences_2), len(next_chars_2), len(sentences)

(4, 4, 4529)

In [102]:
1268+1200+445+195+89+39+10+7+4+1, 4529-3258

(3258, 1271)

In [54]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [55]:
pred = model.predict(X_test, verbose=0)
pred_a1 = pred[0]
pred_t = pred[1]

In [43]:
pred[0].shape, pred[1].shape

((3, 10), (3, 1))

## Evaluate

### Activity

In [32]:
evalAct(next_chars_2, pred[0], targetchartoindice)

Log-loss: 0.123089636443
Accuracy: 100.0%
Top 3 accuracy: 100.0%


### Time

In [33]:
evalTime(next_chars_t_2, pred_t, divisor)

Mean Squared Error: 60581431405.1s	| 701173.974596 days
Mean Absolute Error: 161891.107307s	| 1.87373966791 days
Median Absolute Error: 97399.1495361s	| 1.12730497148 days
